In [ ]:
#install requirements
%pip install -q -r requirements.txt

In [ ]:
# Environment setup
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


In [4]:
import boto3
import os

session = boto3.Session(
    region_name=os.getenv("AWS_REGION", "us-east-1"), profile_name=os.getenv("AWS_PROFILE")
)
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    endpoint_url='https://bedrock-runtime.'+os.getenv('AWS_REGION', 'us-east-1')+'.amazonaws.com',
    )

In [ ]:
import json

prompt = "Hello, world"

body = json.dumps({
    "max_tokens": 4096,
    "temperature": 0.5,
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-haiku-20240307-v1:0")
response_body = json.loads(response.get("body").read())      
print(response_body['content'][0]['text'])

In [7]:
from langchain_community.chat_models import BedrockChat
from langchain.llms.bedrock import Bedrock

def get_llm(model_kwargs, model_id="anthropic.claude-3-haiku-20240307-v1:0"):
    """Creates the LLM object for the langchain conversational bedrock agent.
    Parameters:
        model_kwargs (dict): Dictionary of model_kwargs to be passed to the Bedrock model.
    Returns:
        langchain.llms.bedrock.Bedrock: Bedrock model
    """
    session = boto3.Session(
        region_name=os.getenv("AWS_REGION", "us-east-1"), profile_name=os.getenv("AWS_PROFILE")
    )
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        endpoint_url='https://bedrock-runtime.'+os.getenv('AWS_REGION', 'us-east-1')+'.amazonaws.com',
        )
    if (model_id == "anthropic.claude-3-haiku-20240307-v1:0") or (model_id == "anthropic.claude-3-sonnet-20240229-v1:0"):
        llm = BedrockChat(
            client=bedrock_client,
            model_id=model_id,
            model_kwargs=model_kwargs)
    else:
        llm = Bedrock(
            client=bedrock_client,
            model_id=model_id,
            model_kwargs=model_kwargs)

    return llm

In [ ]:
# Let us test with 2 different models that have different APIs to access to show that Langchain can help abstract this away and always call the model the same way

import time
llm = get_llm(model_id="anthropic.claude-v2:1", model_kwargs={"max_tokens_to_sample": 4096,"temperature": 0.1})
llm_2 = get_llm(model_id="anthropic.claude-3-haiku-20240307-v1:0", model_kwargs={"max_tokens": 4096,"temperature": 0.1})

# For Claude 2.1
start_time = time.time()
response = llm.invoke("Tell me about the top 3 trends in Industrial Manufacturing")
end_time = time.time()
execution_time = end_time - start_time
print("Claude 2.1\n\n" + response)
print(f"\nExecution time: {execution_time:.2f} seconds\n\n")

# For Claude 3 Haiku
start_time2 = time.time()
response2 = llm_2.invoke("Tell me about the top 3 trends in Industrial Manufacturing")
end_time2 = time.time()
execution_time2 = end_time2 - start_time2
print("\n\nClaude 3 Haiku \n\n" + response2.content)
print(f"\nExecution time: {execution_time2:.2f} seconds")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer().fit_transform([response, response2.content])
similarity = cosine_similarity(vectorizer[0:1], vectorizer[1:2])
print(f"Similarity between responses: {similarity[0][0]:.2f}")


In [ ]:
!pip install TextBlob
from textblob import TextBlob

sentiment_1 = TextBlob(response).sentiment
sentiment_2 = TextBlob(response2.content).sentiment
print("Claude 2.1 Sentiment:", sentiment_1)
print("Claude 3 Haiku Sentiment:", sentiment_2)


In [18]:
prompts = [
    "Explain Automation and Robotics",
    "Explain Digital Transformation and Industry 4.0",
    "Explain Sustainability and Environmental Responsibility"
]

results = []
for prompt in prompts:
    start = time.time()
    res = llm.invoke(prompt)
    end = time.time()
    results.append({"prompt": prompt, "response": res, "execution_time": end - start})


In [ ]:
import matplotlib.pyplot as plt

prompts = ["Prompt 1", "Prompt 2", "Prompt 3"]
exec_times = [execution_time, execution_time2]

plt.bar(["Claude 2.1", "Claude 3 Haiku"], exec_times)
plt.xlabel("Models")
plt.ylabel("Execution Time (seconds)")
plt.title("Execution Time Comparison")
plt.show()


In [ ]:
word_count_1 = len(response.split())
word_count_2 = len(response2.content.split())

plt.bar(["Claude 2.1", "Claude 3 Haiku"], [word_count_1, word_count_2])
plt.xlabel("Models")
plt.ylabel("Word Count")
plt.title("Word Count Comparison")
plt.show()


In [ ]:
!pip install textstat

from textstat import flesch_reading_ease

readability_1 = flesch_reading_ease(response)
readability_2 = flesch_reading_ease(response2.content)
print("Readability Score for Claude 2.1:", readability_1)
print("Readability Score for Claude 3 Haiku:", readability_2)


In [ ]:
response_multilingual = llm.invoke("Expliquez les tendances actuelles dans la fabrication industrielle.")
print("Claude 2.1 (French):", response_multilingual)


In [ ]:
prompt_with_examples = """
Example 1: What are the benefits of AI? -> AI enables automation and improves efficiency.
Example 2: Explain cloud computing. -> Cloud computing is the delivery of computing services over the internet.
Now, explain industrial manufacturing trends.
"""
response_few_shot = llm.invoke(prompt_with_examples)
print(response_few_shot)


In [ ]:
long_prompt = "Write a detailed report on the impact of robotics in industrial manufacturing."
long_response = llm.invoke(long_prompt)
print(long_response)
